# Amazon, from web to csv

In [1]:
# %load basic
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.set_option('display.max_columns',200)
pd.options.display.float_format = '{:.2f}'.format
liga = '/home/ef/Documents/Diplomado/Amazon/data/'

In [2]:
import time
start = time.time()

Source:

<https://nijianmo.github.io/amazon/index.html#subsets>

In [3]:
#Importar el código html de la url en cuestión
import requests
import bs4
url_base = 'https://nijianmo.github.io/amazon/index.html#subsets'
res = requests.get(url_base)
soup = bs4.BeautifulSoup(res.text,'lxml')

In [4]:
import re 
#Obtener lista con links que terminen en .json.gz
urls = [re.findall(r'http[\s\S]+\.json\.gz', str(x)) for x in soup.find_all('a')]
#Sólo nos quedamos con el string de los elementos con algún link
urls = [x[0] for x in filter(lambda x:len(x)>0, urls)]
print(len(urls))
urls[:4]

29


['http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/AMAZON_FASHION_5.json.gz',
 'http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/All_Beauty_5.json.gz',
 'http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/Appliances_5.json.gz',
 'http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/Arts_Crafts_and_Sewing_5.json.gz']

In [5]:
#Sólo se trabajará con productos de entretenimiento
entertainment =['Movies and TV','Kindle Store', 
                'Cell Phones and Accessories', 'Video Games'] 

urls = [x for x in urls if 
        re.findall(r'[\w]+(?=_5\.json.gz)',x)[-1].replace('_',' ') in 
        entertainment]

urls

['http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/Cell_Phones_and_Accessories_5.json.gz',
 'http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/Kindle_Store_5.json.gz',
 'http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/Movies_and_TV_5.json.gz',
 'http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/Video_Games_5.json.gz']

In [6]:
import urllib.request
import json
import gzip

def webjson_to_df(url, local_dir = liga, n = 180500):
    #Obtener el nombre de la categoría en minúsculas
    file_name = re.findall(r'[\w]+(?=_5\.json.gz)',url)[-1].lower()
    #Definir dirección para..
    file_dir = f'{local_dir}/raw/{file_name}.json.gz'
    #..descargar el archivo de url y posicionarlo en ella
    urllib.request.urlretrieve(url,file_dir)
    data = []
    #Llamar archivo para cargarlo
    with gzip.open(file_dir) as f:
        for i,l in enumerate(f):
            #Al exceder el número de renglones..
            if i+1 > n:
                #..termina el ciclo
                break
            #De otro modo, sigue compilando los datos
            data.append(json.loads(l.strip()))
    #Finalmente lo convierte a DataFrame
    df = pd.DataFrame.from_dict(data)
    #Crea una columna con la categoría rcorrespondiente
    df['category'] = file_name
    #Y lo exporta como csv
    df.to_csv(f'{local_dir}/csv/{file_name}.csv', index = False)
    #Borra objetos para siguiente iteración
    del file_name, file_dir, data, df

In [7]:
import concurrent.futures

#Ejecuta la función a través de la lista urls (iterable) con todo el power de todos los núcleos
with concurrent.futures.ThreadPoolExecutor() as executor:
    executor.map(webjson_to_df, urls)

In [8]:
import math
def time_exp(x):
    print(str(int(math.floor(x/60))
             ) + " minutos con " + '{:.2f}'.format(60*(x/60 - math.floor(x/60))
                                                  ) + " segundos")

In [9]:
#Tiempo total para correr el notebook
end = time.time()
time_exp(end - start)

25 minutos con 33.83 segundos


In [10]:
#Tono para cuando termina el script
from IPython.lib.display import Audio
import numpy as np

framerate = 4410
play_time_seconds = 1

t = np.linspace(0, play_time_seconds, framerate*play_time_seconds)
audio_data = np.sin(5*np.pi*300*t) + np.sin(2*np.pi*240*t)

#La siguiente línea suena!
Audio(audio_data, rate=framerate, autoplay=True)

##FUNCIÓN SIN CONTADOR, TRAE TODO

import urllib.request
import json
import gzip

def webjson_to_df(url, liga = liga):
    #Obtener el nombre de la categoría en minúsculas
    file_name = re.findall(r'[\w]+(?=_5\.json.gz)',url)[-1].lower()
    #Definir dirección para..
    file_dir = f'{liga}/raw/{file_name}.json.gz'
    #..descargar el archivo de url y posicionarlo en file_dir
    urllib.request.urlretrieve(url,file_dir)
    #Llamar archivo para cargarlo
    with gzip.open(file_dir) as f:
        data = map(lambda l:json.loads(l.strip()),f)
    #Y convertirlo a DataFrame
    df = pd.DataFrame.from_dict(data)
    df['category'] = file_name
    #Y exportarlo
    df.to_csv(liga + file_name + '.csv', index = False)
    #Borrar objetos para siguiente iteración
    del file_name, file_dir, data, df